In [ ]:
import os
import requests
from requests.auth import HTTPBasicAuth
import json
import datetime
from shapely.geometry import shape
from sentinelhub import SHConfig,SentinelHubBYOC, SentinelHubDownloadClient, SentinelHubRequest, DataCollection, MimeType,Geometry, CRS, BBox, bbox_to_dimensions
from sentinelhub_downloader import SentinelHubAPI, Config
import dateutil
from datetime import datetime, timedelta
import numpy as np

from planet import Planet
from planet import Session
from planet.clients import FeaturesClient
import planet.subscription_request
from planet.subscription_request import build_request

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import geopandas as gpd
from pyproj import Transformer
from datetime import datetime, timedelta


In [ ]:

# --- User Inputs ---

# Collection ID of the Feature Collection with a single AOI, with quota reserved for
# the PV you're generating. 
PLANET_FEATURE_COLLECTION_ID = ""

# Your Planet API Key. You can get this from https://www.planet.com/account/#/user-settings 
# or if you have done `pip install planet` then you can type `planet auth value` and
# copy the results.
PLANET_API_KEY = ""

# Sentinel Hub Client ID and Secret. Go to https://apps.sentinel-hub.com/dashboard/#/account/settings
# and hit '+ Create' under OAuth Clients. Be sure to store these, you won't see 'secret' again.
SENTINEL_HUB_CLIENT_ID = ""
SENTINEL_HUB_CLIENT_SECRET = ""

# This name is used to identify the subscription, customize it to be a meaningful name
# so you can easily identify it later.
SUBSCRIPTION_BASE_NAME = ""


# Customize these values for the PV you want to download.
PV_VAR_TYPE="biomass_proxy"
PV_VAR_ID="BIOMASS-PROXY_V4.0_10"
# Other common options for SWC are: SWC-AMSR2-C_V2.0_100, SWC-AMSR2-X_V2.0_100, SWC-SMAP-L_V5.0_1000
# Full list of options: https://developers.planet.com/docs/planetary-variables/soil-water-content-technical-specification/

# Bands
BANDS="CB"

# Start date for the data you want to download.
SUB_START_TIME = "2024-04-01"

# Comment out either an end date or NO_END_TIME to make an ongoing subscription
SUB_END_TIME = "2024-04-07"
#SUB_END_TIME = "NO_END_TIME"

# Define output folder for TIFF files
# Change this to the path of the folder you want to save the TIFF files to
OUTPUT_FOLDER = "D:/Bahram/Planet/DL/"

#Locate your gdal install for your python setup. It's probably wherever your environment is ../share/gdal
os.environ['GDAL_DATA'] = 'D:\\Conda\\GEE\\share\\gdal'

#Path to your local shapefile matching what you uploaded to planet
#It will use its exact geometry, so it needs to be the same
GeometryFile = ''
OutGeometryFile = ''

#Needs to be 326(UTM Zone)
#I think
GeometryCRS = 32614

In [ ]:
GeometryFile = 'D:\\Bahram\\Planet\\Planet_MB_Sites_Small.shp'
GeometryData = gpd.read_file(GeometryFile)
GeometryData = GeometryData.to_crs(32614)

In [ ]:
def sentinelhub_compliance_hook(response):
    response.raise_for_status()
    return response

# Create OAuth2 session for machine-to-machine auth (Client Credentials)
client = BackendApplicationClient(client_id=SENTINEL_HUB_CLIENT_ID)
oauth = OAuth2Session(client=client)

# Register hook to avoid misleading error messages
oauth.register_compliance_hook("access_token_response", sentinelhub_compliance_hook)

# Fetch access token
token = oauth.fetch_token(
    token_url='https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token',
    client_secret=SENTINEL_HUB_CLIENT_SECRET,
    include_client_id=True
)

# Make an authenticated request
resp = oauth.get("https://services.sentinel-hub.com/configuration/v1/wms/instances")

In [ ]:
##defining base url
subscriptions_url = "https://api.planet.com/subscriptions/v1"

plsdk_auth = "YOUR_PLANET_API_KEY"
auth = planet.Auth.from_key(key=PLANET_API_KEY)
sess = planet.Session(auth)
pl = planet.Planet(sess)

In [ ]:
GeometryData = gpd.read_file(GeometryFile)
GeometryData = GeometryData.to_crs(GeometryCRS)

In [ ]:
####################################################
#--------------------------------------------------#
# Wait! Did you already create your subscriptions? #
# If so, skip the next step and instead simply set #
#   your COLLECTION_ID value. This is visible in   #
#        your sentinelhub account dashboard        #
####################################################
#--------------------------------------------------#

In [ ]:
#Function to create bulk subscriptions for multi-polygon collections

def create_planet_subscription_bulk():
    start_time = datetime.strptime(SUB_START_TIME, "%Y-%m-%d")
    end_time = datetime.strptime(SUB_END_TIME, "%Y-%m-%d")
    
    subscription_name = SUBSCRIPTION_BASE_NAME
    
    pv_source = planet.subscription_request.subscription_source(
    source_id=PV_VAR_ID,
    geometry={
        "content": "pl:features/my/" + PLANET_FEATURE_COLLECTION_ID,
        "type": "ref",
    },
    start_time=start_time,
    end_time=end_time,
)
    
    payload = planet.subscription_request.build_request(
        name=subscription_name,
        source=pv_source,
        hosting = planet.subscription_request.sentinel_hub(
             collection_id="",
             create_configuration=True
        ))
    
    subs = [payload]
    
    try:
        result = pl.subscriptions.bulk_create_subscriptions(subs)
        print(result)
        return result
    except Exception as e:
        print(f"❌ Error creating subscription '{subscription_name}': {str(e)}")
        raise

# Create subscriptions for each item
subreq = create_planet_subscription_bulk()


In [ ]:
#If you've just run the subscription request, you will need to wait for it to process first
#Once complete, you can access the ID from here
headers = {"Authorization": f"api-key {PLANET_API_KEY}"}
resp = requests.get(subreq['_links']['list'], headers=headers)
subs = resp.json()


#All items will share the same collection ID, so we can just read into the first one to get it
COLLECTION_ID = subs['subscriptions'][0]['hosting']['parameters']['collection_id']

In [ ]:
#Alternatively, manually enter the collection ID if you have previously completed this step and
#do not wish to create another request
COLLECTION_ID = 'b7b867c8-dc83-4ac8-b2ab-8b4a4677453d'

In [ ]:
#Create nested lists and extract stats from each of the dates and polygons.
#Remember that this will iterate based on the shapefile, not the collection. If you want to extract a subset
#of the collection, just use a subset in the shapefile

#It's likely you will need to modify the evalscript and stats request significantly to fit your data
#At the very least, you will probably have to adjust the bands

InitDate = datetime.strptime(SUB_START_TIME, "%Y-%m-%d")
EndDate = datetime.strptime(SUB_END_TIME, "%Y-%m-%d")

TimeLen = (EndDate - InitDate).days

Mins = [[[None] for x in range(len(GeometryData['geometry']))] for y in range(TimeLen)]
Maxes = [[[None] for x in range(len(GeometryData['geometry']))] for y in range(TimeLen)]
Means = [[[None] for x in range(len(GeometryData['geometry']))] for y in range(TimeLen)]
StDev = [[[None] for x in range(len(GeometryData['geometry']))] for y in range(TimeLen)]
SampleCount = [[[None] for x in range(len(GeometryData['geometry']))] for y in range(TimeLen)]

for j in range(0, len(GeometryData['geometry'])):

    TileGeojson = GeometryData['geometry'][j].__geo_interface__
    
    resolution = 10

    evalscript = """
    //VERSION=3
    function setup() {
      return {
        input: [{
          bands: [
            "CB",
            "dataMask"
          ]
        }],
        output: [
          {
            id: "output_CB",
            bands: 1,
            sampleType: "FLOAT32"
          },
          {
            id: "dataMask",
            bands: 1
          }]
      }
    }
    function evaluatePixel(samples) {
        return {
            output_CB: [samples.CB],
            dataMask: [samples.dataMask]
            }
    }
    """

    stats_request = {
      "input": {
        "bounds": {
          "geometry": TileGeojson,
        "properties": {
            "crs": "http://www.opengis.net/def/crs/EPSG/0/" + str(GeometryCRS)
            }
        },
        "data": [
          {
            "type": "byoc-" + COLLECTION_ID,
            "dataFilter": {
                "mosaickingOrder": "leastRecent"
            },
          }
        ]
      },
      "aggregation": {
        "timeRange": {
                "from": SUB_START_TIME + "T00:00:00Z",
                "to": SUB_END_TIME + "T00:00:00Z"
          },
        "aggregationInterval": {
            "of": "P1D"
        },
        "evalscript": evalscript,
        "resx": resolution,
        "resy": resolution
      }
    }


    headers = {
      'Content-Type': 'application/json',
      'Accept': 'application/json'
    }

    url = "https://services.sentinel-hub.com/api/v1/statistics"
    response = oauth.request("POST", url=url , headers=headers, json=stats_request)
    sh_statistics = response.json()

    #Assign response values to lists
    for i in range(0, len(sh_statistics['data'])):
        Mins[i][j] = sh_statistics['data'][i]['outputs']['output_CB']['bands']['B0']['stats']['min']
        Maxes[i][j] = sh_statistics['data'][i]['outputs']['output_CB']['bands']['B0']['stats']['max']
        Means[i][j] = sh_statistics['data'][i]['outputs']['output_CB']['bands']['B0']['stats']['mean']
        StDev[i][j] = sh_statistics['data'][i]['outputs']['output_CB']['bands']['B0']['stats']['stDev']
        SampleCount[i][j] = sh_statistics['data'][i]['outputs']['output_CB']['bands']['B0']['stats']['sampleCount']
        

In [ ]:
#Update the geodataframe and save to new file
for i in range(len(Mins)):
    UpdateDate = InitDate + timedelta(days = i)
    UpdateDate = datetime.strftime(UpdateDate, "%Y%m%d")
    
    GeometryData[UpdateDate + "Min"] = Mins[i]
    GeometryData[UpdateDate + "Max"] = Maxes[i]
    GeometryData[UpdateDate + "Mean"] = Means[i]
    GeometryData[UpdateDate + "StDev"] = StDev[i]
    GeometryData[UpdateDate + "n"] = SampleCount[i]
    
GeometryData.to_file(OutGeometryFile)